In [1]:
import utils
import numpy as np
import pandas as pd

In [2]:
INPUT_PATH = "test_results/test_neurons_results.csv"

neuron_results_df = pd.read_csv(INPUT_PATH)
neuron_results_df

,Unnamed: 0,layer,neuron,explainer,NeuronViewer,Original,Summary,Highlight,HighlightSummary,AVHS
0,0,0,286,gpt-3.5-turbo,words related to comparison.,comparisons between different entities or situ...,words or phrases that indicate a comparison or...,words related to comparisons or contrasts betw...,words related to comparisons or contrasting.,words related to comparing or contrasting two ...
1,1,10,1838,gpt-3.5-turbo,phrases describing positions or situations in...,phrases related to personal situations or expe...,"reflexive pronouns and their variations, such ...","instances of reflexive pronouns (e.g. ""themsel...",pronouns or words related to self or individua...,pronouns or words related to self-identificati...
2,2,20,193,gpt-3.5-turbo,verbs indicating questioning or challenging be...,phrases related to calling out or criticizing ...,words related to expressing criticism or quest...,tokens related to expressing or questioning op...,"words related to expressing disagreement, crit...",instances where someone is being called out or...
3,3,30,1685,gpt-3.5-turbo,expressions of gratitude and agreeing to rece...,phrases related to newsletter subscriptions an...,words and phrases related to subscribing to ne...,phrases related to expressing gratitude or tha...,words and phrases related to expressing gratit...,tokens related to newsletter sign-up and subsc...
4,4,40,431,gpt-3.5-turbo,"numbers related to time, dates, and measureme...",timestamps and time-related information in text.,numbers and time-related information.,"time-related information, such as timestamps o...","time-related information, including hours, min...","information related to dates, times, and events."


In [3]:
def get_embedding(text_to_embed):
	# Embed a line of text
	response = utils.embedding_with_backoff(
    	model= "text-embedding-ada-002",
    	input=text_to_embed
	)
	# Extract the AI output embedding as a list of floats
	embedding = response["data"][0]["embedding"]

	return embedding

def cosSim(A, B):
  A=np.array(A).flatten()
  B=np.array(B).flatten()
  return np.dot(A,B)/(np.linalg.norm(A)*np.linalg.norm(B))

In [4]:
modes = ["Original", "Summary", "Highlight", "HighlightSummary", "AVHS"]
AdaCS_results = {"layer":[], "neuron":[]}
for mode in modes:
    AdaCS_results[mode] = []

for i, row in neuron_results_df.iterrows():
    layer = row["layer"]
    neuron = row["neuron"]
    NV_Explan = row["NeuronViewer"]
    AdaCS_results["layer"].append(layer)
    AdaCS_results["neuron"].append(neuron)
    
    for mode in modes:
        prompt_Explan_Vector = get_embedding(row[mode])
        NV_Explan_Vector = get_embedding(NV_Explan)
        AdaCS_results[mode].append(cosSim(prompt_Explan_Vector, NV_Explan_Vector))

new_df = pd.DataFrame(AdaCS_results)
new_df

,layer,neuron,Original,Summary,Highlight,HighlightSummary,AVHS
0,0,286,0.838150,0.871811,0.910371,0.925517,0.908650
1,10,1838,0.885322,0.831552,0.841508,0.849710,0.852272
2,20,193,0.846874,0.904602,0.859872,0.871175,0.802579
3,30,1685,0.798638,0.826908,0.884671,0.891365,0.790127
4,40,431,0.852980,0.911374,0.859724,0.867457,0.874474


In [5]:
new_df.to_csv(INPUT_PATH.split(".")[0] + "_adaCS.csv")

In [6]:
for mode in modes:
    avg = np.mean(new_df[mode])
    print(mode, avg)

Original 0.8443929253179865
Summary 0.8692494952946233
Highlight 0.8712292357118165
HighlightSummary 0.8810450174990387
AVHS 0.8456201711838094
